In [6]:
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import copy
import math
import numpy as np
from torchvision import datasets, transforms
import torch
from numpy import *

from utils.sampling import mnist_iid_cluster, mnist_noniid_cluster, cifar_noniid_cluster,cifar_noniid_cluster_varying_users
from models_v4.Update import LocalTrain,ClusterDetect
from models_v4.Nets import  CNNMnist, CNNCifar,  CNNMnist2
from models_v4.Fed import FedAvg_vectorization

from models_v4.test import test_acc
from scipy.linalg import null_space


%load_ext autoreload
%autoreload 2
class my_argument:    
    epochs = 10   #"rounds of training"
    num_users = 5 # "number of users: K"
    frac = 0.1 #"the fraction of clients: C"
    local_ep=5 #"the number of local epochs: E"
    local_bs=50 #"local batch size: B"
    bs=128 #"test batch size"
    lr=0.0001 #"learning rate"
    momentum=0.5 # "SGD momentum (default: 0.5)"
    split='user' # "train-test split type, user or sample"

    # model arguments
    model = 'cnn'
    
    # other arguments
    dataset='cifar' #, help="name of dataset")
    iid=0
    num_classes=10#, help="number of classes")
    num_channels=3#, help="number of channels of images")
    gpu=3#, help="GPU ID, -1 for CPU")
    stopping_rounds=10#, help='rounds of early stopping')
    verbose='False'#, help='verbose print')
    seed=1#, help='random seed (default: 1)')
    cluster=5
    opt='ADAM'
args = my_argument()
#args.device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() and args.gpu != -1 else 'cpu')
print(args.epochs)
np.random.seed(1)


10


In [7]:
encoder1=[]
encoder2=[]
decoder1=[]
decoder2=[]
A_bar1=[]
A_bar2=[]
U1=[]
U2=[]
mu=0
sigma=1
#alpha=0.001
for i in range(args.cluster):
    encoder1.append([])
    decoder1.append([])
    encoder2.append([])
    decoder2.append([])
    A_bar1.append([])
    A_bar2.append([])
    U1.append([])
    U2.append([])
    decoder1.append([])
    decoder2.append([])
part=1000; # the size of vector that we want to send each time
total_size=62006 #total length of the vector
d1=part
slot=math.ceil(total_size/part) # the no. of slots we need to send vectors
d2=total_size%part
slot1=math.ceil(total_size/d1)-1
slot2=total_size%d1
print(slot)
print(d2)
K=args.cluster
for i in range(args.cluster):
    num1=np.random.normal(mu, sigma, K*d1*d1)
    #num1=np.random.rand(K*d1*d1)
    num2=np.random.normal(mu, sigma, K*d2*d2)
    #num2=np.random.rand(K*d2*d2)
    J1=np.random.rand(d1,d1)
    for j in range(K):
        if (j!=i):
            encoder=np.zeros((d1,d1))
            J1=np.vstack([J1,encoder])
        else:
            encoder=np.identity(d1)
            J1=np.vstack([J1,encoder])
    J1 = np.delete(J1,np.s_[0:d1], axis=0)
    encoder1[i]=J1
    #encoder2[i]=num2.reshape(K*d2,d2)
    #divide the vector into 15 parts of size 10000 each and one part of size 9010
#define encoder[0],encoder[1],encoder[2],encoder[3]
K=args.cluster
print(encoder1[1].shape)
#N_R=5#no of receiver antenna
for i in range(K): # K= no. of clusters
    J1=np.random.rand(K*d1,d1)
    #J2=np.random.rand(K*d2,d2)
    #print(type(J2))
    print(type(encoder1[i]))
    for j in range(args.cluster):
        if(j!=i):
            J1=np.hstack([J1,encoder1[j]]) #concatenating different encoder matrices
            #J1.append(encoder1[j])
            #J2=np.hstack([J2,encoder2[j]])
    print(J1.shape)
    J1 = np.delete(J1,np.s_[0:d1], axis=1)# delete the initial d rows in J
    print(J1.shape)
    #J2 = np.delete(J2,np.s_[0:d2], axis=1)
    #print(J2.shape)
    A_bar1[i]=J1
    #A_bar2[i]=J2
    print(A_bar1[i].shape)
    #print(A_bar2[i].shape)
    print("printing type of A_bar")
    print(type(A_bar1[i]))
    U1[i]=null_space((A_bar1[i].transpose()))
    print(U1[i].shape)
    #U2[i]=null_space((A_bar2[i].transpose()))
    #print(U2[i].shape)
    decoder1[i]=(np.linalg.inv((U1[i].transpose())@(encoder1[i])) @(U1[i].transpose()))
    #decoder2[i]=(np.linalg.inv((U2[i].transpose())@(encoder2[i])) @(U2[i].transpose()))
    #decoder1[i]=
    print("printing size of decoder1")
    print(decoder1[0].shape)
    print(type(decoder1))
    #print("printing size of decoder2")
    #print(decoder2[0].shape)


63
6
(5000, 1000)
<class 'numpy.ndarray'>
(5000, 5000)
(5000, 4000)
(5000, 4000)
printing type of A_bar
<class 'numpy.ndarray'>
(5000, 1000)
printing size of decoder1
(1000, 5000)
<class 'list'>
<class 'numpy.ndarray'>
(5000, 5000)
(5000, 4000)
(5000, 4000)
printing type of A_bar
<class 'numpy.ndarray'>
(5000, 1000)
printing size of decoder1
(1000, 5000)
<class 'list'>
<class 'numpy.ndarray'>
(5000, 5000)
(5000, 4000)
(5000, 4000)
printing type of A_bar
<class 'numpy.ndarray'>
(5000, 1000)
printing size of decoder1
(1000, 5000)
<class 'list'>
<class 'numpy.ndarray'>
(5000, 5000)
(5000, 4000)
(5000, 4000)
printing type of A_bar
<class 'numpy.ndarray'>
(5000, 1000)
printing size of decoder1
(1000, 5000)
<class 'list'>
<class 'numpy.ndarray'>
(5000, 5000)
(5000, 4000)
(5000, 4000)
printing type of A_bar
<class 'numpy.ndarray'>
(5000, 1000)
printing size of decoder1
(1000, 5000)
<class 'list'>


In [8]:
# load dataset and split users
u=[4,5,8,12,15]
#u=[5,5,5,5,5]
u=[2,5,5,6,7]
u=[2,3,4,4,12]
u=[5,5,5,5,5]
if args.dataset == 'mnist':
    trans_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
    dataset_train = datasets.MNIST('./data/mnist/', train=True, download=True, transform=trans_mnist)
    dataset_test = datasets.MNIST('./data/mnist/', train=False, download=True, transform=trans_mnist)
    count=0
    #print(len(dataset_train))
    dict_users=[] #2D array in each row, users of a particular cluster
    train_data=[]
    test_data=[]
    for j in range(args.cluster):
        train_data.append([])
        test_data.append([])
        dict_users.append([])
    for j in range(len(dataset_train)):
        data,label=j
        if (label==0) | (label==1):
            train_data[0].append(dataset_train[j])
        elif (label==2) | (label==3):
            train_data[1].append(dataset_train[j])
        elif (label==4) | (label==5):
            train_data[2].append(dataset_train[j])
        elif (label==6) | (label==7):
            train_data[3].append(dataset_train[j])
        elif (label==8) | (label==9):
            train_data[4].append(dataset_train[j])
    for j in range(len(dataset_test)):
        data,label=j
        if (label==0) | (label==1):
            test_data[0].append(dataset_test[j])
        elif (label==2) | (label==3):
            test_data[1].append(dataset_test[j])
        elif (label==4) | (label==5):
            test_data[2].append(dataset_test[j])
        elif (label==6) | (label==7):
            test_data[3].append(dataset_test[j])
        elif (label==8) | (label==9):
            test_data[4].append(dataset_test[j])
    
#defining 5 different types of datasets for 5 different clusters
    
    if args.iid:
        for cluster_no in range(args.cluster):
            dict_users[cluster_no] = mnist_iid_cluster(train_data[cluster_no], args.num_users)
    else:
        for cluster_no in range(args.cluster):
            dict_users[cluster_no] = mnist_noniid_cluster(train_data[cluster_no], args.num_users)
elif args.dataset == 'cifar':
    trans_cifar = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    dataset_train = datasets.CIFAR10('./data/cifar', train=True, download=True, transform=trans_cifar)
    dataset_test = datasets.CIFAR10('./data/cifar', train=False, download=True, transform=trans_cifar)
#defining 5 different types of datasets for 5 different clusters
    count=0
    #print(len(dataset_train))
    dict_users=[] #2D array in each row, users of a particular cluster
    train_data=[]
    test_data=[]
    for j in range(args.cluster):
        train_data.append([])
        test_data.append([])
        dict_users.append([])
    for j in range(len(dataset_train)):
        data,label=dataset_train[j]
        if (label==0) | (label==1):
            train_data[0].append(dataset_train[j])
        elif (label==2) | (label==3):
            train_data[1].append(dataset_train[j])
        elif (label==4) | (label==5):
            train_data[2].append(dataset_train[j])
        elif (label==6) | (label==7):
            train_data[3].append(dataset_train[j])
        elif (label==8) | (label==9):
            train_data[4].append(dataset_train[j])
    for j in range(len(dataset_test)):
        data,label=dataset_test[j]
        if (label==0) | (label==1):
            test_data[0].append(dataset_test[j])
        elif (label==2) | (label==3):
            test_data[1].append(dataset_test[j])
        elif (label==4) | (label==5):
            test_data[2].append(dataset_test[j])
        elif (label==6) | (label==7):
            test_data[3].append(dataset_test[j])
        elif (label==8) | (label==9):
            test_data[4].append(dataset_test[j])

    if args.iid:
        for cluster_no in range(args.cluster):
            dict_users[cluster_no] = cifar_iid_cluster(train_data[cluster_no], u[cluster_no])
    else:
        for cluster_no in range(args.cluster):
            dict_users[cluster_no] = cifar_noniid_cluster_varying_users(train_data[cluster_no], u[cluster_no])
else:
    exit('Error: dataset not found')
img_size = dataset_train[0][0].shape
#print(dict_users[0])
#print((dict_users[0][4]))
print(len(dataset_train))
print(len(dict_users))
print(len(train_data[0]))
#print(train_data[0])
#idxs_users = np.random.choice(range(args.num_users), m, replace=False)
#print(idxs_users)

Files already downloaded and verified
Files already downloaded and verified
50000
5
10000


In [9]:
print(dict_users[1][1])

[3747 8142 1802 ... 4528 9021 5676]


In [10]:
print(len(train_data[1]))

10000


In [11]:
use_cuda = torch.cuda.is_available()
#print(use_cuda)
args.device = torch.device("cuda:3" if use_cuda else "cpu")
#args.device=torch.device("cpu")
print(args.device)

cuda:3


In [12]:
acc_test=[]
acc_test_arr=[]
loss_test=[]
loss_test_arr=[]
for cluster_no in range(args.cluster):
    acc_test.append([])
    loss_test.append([])
    acc_test_arr.append([])
    loss_test_arr.append([])
for cluster_no in range(args.cluster):
    for i in range(args.cluster):
        acc_test_arr[cluster_no].append([])
        loss_test_arr[cluster_no].append([])
print(acc_test_arr)
noise_acc=[]
for user in range(25):
    noise_acc.append([])

[[[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []]]


In [13]:
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        seed=123
        torch.cuda.manual_seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        np.random.seed(seed)
        random.seed(seed)
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        nn.init.xavier_uniform(m.weight.data, nn.init.calculate_gain('relu'))
        #nn.init.xavier_uniform(m.bias.data)
        torch.nn.init.zeros_(m.bias.data)
    if isinstance(m, torch.nn.Linear):
        seed=123
        torch.cuda.manual_seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        np.random.seed(seed)
        random.seed(seed)
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        torch.nn.init.xavier_uniform_(m.weight.data)
        #torch.nn.init.xavier_uniform_(m.bias.data)
        torch.nn.init.zeros_(m.bias.data)
    if isinstance(m, nn.BatchNorm2d):
        seed=123
        torch.cuda.manual_seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        np.random.seed(seed)
        random.seed(seed)
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        torch.nn.init.xavier_uniform_(m.weight.data)
        #torch.nn.init.xavier_uniform_(m.bias.data)
        torch.nn.init.zeros_(m.bias.data)
        #conv1.bias.data.fill_(0.01)
        

In [14]:
# build model
from models_v4.Fed import weight_vectorization,FedSubstract,FedAvg_gradient,FedAdd
from models_v4.Fed import FedAdd,FedSubstract,weight_vectorization_gen,FedAvg_gradient
import numpy as np
import copy
if args.model == 'cnn' and args.dataset == 'cifar':
    net_glob=[]
    for i in range(args.cluster):
        net_glob.append(CNNCifar(args=args).to(args.device))
elif args.model == 'cnn' and args.dataset == 'mnist':
    net_glob=[]
    for i in range(args.cluster):
        net_glob.append(CNNMnist2(args=args).to(args.device))

else:
    exit('Error: model not found')
#print(net_glob)
w_glob=[]
# copy weights
for i in range(args.cluster):
    net_glob[i].train()
    w_glob.append(net_glob[i].state_dict())
abs_vect,layer_size=weight_vectorization_gen(w_glob[2])
net_glob_in=copy.deepcopy(net_glob)
w_glob_in=copy.deepcopy(w_glob)
print(len(abs_vect))
#print("weights")
#print(w_glob)
#print(w_glob.shape)
#print(w_glob[2])

62006


In [15]:
w_glob_in[0]=torch.load("net1_2.pt")
w_glob_in[1]=torch.load("net2_2.pt")
w_glob_in[2]=torch.load("net3_2.pt")
w_glob_in[3]=torch.load("net4_2.pt")
w_glob_in[4]=torch.load("net5_2.pt")
net_glob_in[0].load_state_dict(w_glob_in[0])
net_glob_in[1].load_state_dict(w_glob_in[1])
net_glob_in[2].load_state_dict(w_glob_in[2])
net_glob_in[3].load_state_dict(w_glob_in[3])
net_glob_in[4].load_state_dict(w_glob_in[4])

<All keys matched successfully>

In [16]:
# build model
d=62006
from models_v4.Fed import weight_vectorization_cifar,FedAdd,FedSubstract,FedAvg_gradient
import numpy as np
import copy
slot=math.ceil(d/d1) # the no. of slots we need to send vectors
w_glob=copy.deepcopy(w_glob_in)
net_glob=copy.deepcopy(net_glob_in)
#u=[5,8,10,12,15]
#u=range(0,50)
idx_users=range(0,25)
# training
cv_loss, cv_acc = [], []
val_loss_pre, counter = 0, 0
net_best = None
best_loss = None
val_acc_list, net_list = [], []
num=[]
import random
D=part #dimension of noise vector = Nr= Kd
mu=0
sigma=1
K=args.cluster

#hist_ = np.zeros(10,dtype=int)
sample=0 # fro the purpose of using fresh samples in each iteration
args.lr=0.0001
for iter in range(500): #args.epochs
    print("iteration number",iter)
    if(iter%4==0):
        sample=0
    encoded=[]
    decoded=[]
    num=[]
    noise=[] #noise vector
  
    num=np.random.normal(mu, sigma, K*D)
    num=np.transpose(num.reshape(1,len(num)))
    #m = 10
    loss_train=[]
    cluster_block=[]
    noise_dec=[] #decoded noise vector
    for cluster_no in range(args.cluster):
        loss_train.append([])
        cluster_block.append([])
        decoded.append([])
        noise_dec.append([])
    Power=[]
    Power2=[]
    alpha=[]
    feature_vector2=[]
    for i in range(args.cluster):
        Power.append([])
        Power2.append([])
        alpha.append([])
        feature_vector2.append(np.zeros((d,1)))
                
    #idxs_users = np.random.choice(range(args.num_users), m, replace=False)
    idx_users=[]
    sorted_train_data=[]
    sorted_test_data=[]
    for cluster_no in range(args.cluster):
        for index in dict_users[cluster_no]:
            idx_users.append(index) # putting the data indices of users in this list
            sorted_train_data.append(train_data[cluster_no])#putting the corresponding training data in this array
            sorted_test_data.append(test_data[cluster_no])
            
    for user in range(len(idx_users)): # no. of loop= no. of users
        cluster_loss=[]
        L=len(idx_users[user])
        L=600
        sample_size=int(L/8)
        #sample_size=250
        for i in range(args.cluster):
             # each type of dataset belong to 5 users
            local2 = ClusterDetect(args=args, dataset= sorted_train_data[user], idxs=idx_users[user][(sample*sample_size):(sample+1)*sample_size])
            #local2 = ClusterDetect(args=args, dataset= sorted_train_data[user], idxs=idx_users[user][0:599])
            # using 1st 600 data
            w2, loss2 = local2.train2(net=copy.deepcopy(net_glob[i]).to(args.device))
            cluster_loss.append(loss2)
        #print("printing user")
        #print(user)
        #print(cluster_loss)
        minimum=min(cluster_loss)
        index_of_min=cluster_loss.index(minimum)
        cluster_block[index_of_min].append(user)
    print(cluster_block[0])
    print(cluster_block[1])
    print(cluster_block[2])
    print(cluster_block[3])
    print(cluster_block[4])
    K_global=part
    m_global=np.zeros((d,1))
    location_global=np.random.choice(range(d),K_global,replace=False)
    for i in location_global:
        m_global[i]=1
    updated=[]
    model_diff=[]
    grad_vect=[]
    prev=[]
    error=[]
    grad_vect_send=[]
    store_grad=[]
    for i in range(25):
        updated.append([])
        model_diff.append([])
        grad_vect.append([])
        prev.append([])
        error.append(np.zeros((d,1)))
        grad_vect_send.append([])
        store_grad.append([])
    b=part  
    sigma_R=1/math.sqrt(b)
    mu_R=0
    R=np.random.normal(mu_R, sigma_R, (b,total_size))
    superposition=0
    for cluster_no in range(args.cluster):
        w_locals, loss_locals,grad_locals,diff_locals= [],[],[],[]
        if(cluster_block[cluster_no]==[]):
            continue
        for user2 in cluster_block[cluster_no]:
            trans=np.array([])
            #local = LocalUpdate(args=args, dataset=sorted_train_data[user], idxs=idx_users[user][(sample+1)*600:(sample+2)*600])
            #local = LocalUpdate(args=args, dataset=sorted_train_data[user], idxs=idx_users[user][600:1199])
            #print(user2)
            total_P=0
            updated[user2]=copy.deepcopy(w_glob[cluster_no])
            L=len(idx_users[user2])
            L=600
            sample_size=int(L/8)
            #sample_size=250
            local = LocalTrain(args=args, dataset=sorted_train_data[user2], idxs=idx_users[user2][(sample+1)*sample_size:(sample+2)*sample_size])
            #using 2nd half data
            w, loss = local.train(net=copy.deepcopy(net_glob[cluster_no]).to(args.device))
            w_locals.append(copy.deepcopy(w))
            loss_locals.append(copy.deepcopy(loss))
            prev[user2]=updated[user2]
            model_diff[user2]=FedSubstract(w,prev[user2])
            grad_vect[user2],layer_size=weight_vectorization_gen(model_diff[user2]) # vectorizing the gradient
            #grad_vect[user2]=grad_vect[user2]+error[user2] # error feedback
            array_one=np.ones((d,1))
            model_vector,layer_size=weight_vectorization_gen(w)
            M=max(abs(model_vector))
            mask=m_global
            count=0
            #calculating the modified gradient to be sent to server
            #grad_vect_send[user2]=np.multiply(mask,grad_vect[user2])
            grad_vect_send[user2]= R@grad_vect[user2]
            #error[user2]=grad_vect[user2]-grad_vect_send[user2]
            grad_locals.append(grad_vect_send[user2])
            store_grad[user2]=grad_vect_send[user2]
            diff_locals.append(copy.deepcopy(model_diff[user2]))
            #print(M)
            SNR=math.log10(M**2)
            #print(SNR)
            #n=K*K*d1*d1
            #H = np.random.normal(loc=0, scale=np.sqrt(2)/2, size=(n, 2)).view(np.complex128)
            #H=H.reshape(K*d1,K*d1)
            #U, D, VT = np.linalg.svd(H)
            #HT=np.matrix(H).getH()
            #left_inv=np.linalg.inv(HT@H)@HT
            #E1= left_inv@(encoder1[cluster_no])
            #for i in range(slot1):
                #z=(abs(np.array(E1 @ model_vector[i*d1:(i+1)*d1])))**2
                #trans=np.concatenate((trans,z[:,0]))
                #total_P=total_P+sum(z)
            d2=total_size%d1
            n=K*K*d2*d2
            #H = np.random.normal(loc=0, scale=np.sqrt(2)/2, size=(n, 2)).view(np.complex128)
            #H=H.reshape(K*d2,K*d2)
            #U, D, VT = np.linalg.svd(H)
            #HT=np.matrix(H).getH()
            #left_inv=np.linalg.inv(HT@H)@HT
#print(left_inv.shape)
            #E2= left_inv@(encoder2[cluster_no])
            #z=(abs(np.array(E2 @ model_vector[slot1*d1:slot1*d1+d2])))**2
            #trans=np.concatenate((trans,z[:,0]))
            #M=max(abs(trans))
            #print(M)
            #total_P=total_P + sum(z)
            #print(total_P)
            Power[cluster_no].append(total_P)
        Power2[cluster_no]=(sum(Power[cluster_no]))/len(cluster_block[cluster_no])
        P_limit=2000000000
        #alpha[cluster_no]=math.sqrt(P_limit/Power2[cluster_no])
        alpha[cluster_no]=1000*100
        grad_avg=FedAvg_gradient(grad_locals)
        loss_avg = sum(loss_locals) / len(loss_locals)
        loss_train[cluster_no].append(loss_avg)
        #print(len(w_locals))
        j=w_locals[0]
        feature_vector=grad_avg
        #print(len(feature_vector))
        #print(feature_vector)
        #fecture vector is dx1 vector after flattening all types of weight matrices and concatenaitng them
        increment=0 #keeps track of feature_vactor index
        superposition=superposition+alpha[cluster_no]*encoder1[cluster_no]@feature_vector
    received=superposition
    #print(len(superposition))
    #superposition=superposition+num
    #print("total received Power")
    print(sum((received)**2))
    #print("received min max")
    #print(max(abs(received)))
    #print(min(abs(received)))
    m_global=np.zeros((d,1))
        
    for cluster_no in range(args.cluster):
        #print(w_glob[cluster_no])
        if(cluster_block[cluster_no]==[]):
            continue
        flat=[]
        for i in range(16): # 4 layers in parameter
            flat.append([])
        increment=0
        decoded[cluster_no]=(1/alpha[cluster_no])*decoder1[cluster_no]@superposition
        count=0
    
        decoded[cluster_no]=R.transpose()@decoded[cluster_no]
        count=0
        w_glob_prev=copy.deepcopy(w_glob[cluster_no])
        for i in range(len(w_glob[cluster_no].keys())): # 4 layers in parameter
            flat.append([])

        for h in w_glob_prev.keys():
            s=list(w_glob[cluster_no][h].shape)
            if (len(s)==0):
                new=np.array(0)
                decoded[cluster_no]=np.delete(decoded[cluster_no],np.s_[0])
            else:
                z=np.prod(list(w_glob[cluster_no][h].shape))
                flat[count]=decoded[cluster_no][0:z] # taking out the vector for the specified layer
                decoded[cluster_no]=np.delete(decoded[cluster_no],np.s_[0:z])# deleting that vector from decoded after taking out
             
                new=flat[count].reshape(list(w_glob[cluster_no][h].shape)) #reshaping back to the marix
              
            w_glob[cluster_no][h]=torch.from_numpy(new) #converting the matrix to a tensor
            #print(w_glob[cluster_no][h].shape)
            count=count+1
    # update global weights
        
        global_diff = w_glob[cluster_no]
        w_glob[cluster_no]=FedAdd(w_glob_prev,global_diff)
        net_glob[cluster_no].load_state_dict(w_glob[cluster_no])
        #printing loss in each iteration
        for i in range(args.cluster):
            acc_test[i], loss_test[i] = test_acc(net_glob[cluster_no], test_data[i], args)
            acc_test_arr[i][cluster_no].append(acc_test[i])
            loss_test_arr[i][cluster_no].append(loss_test[i])
        for user in cluster_block[cluster_no]:
            acc,loss= test_acc(net_glob[cluster_no], sorted_test_data[user], args)
            noise_acc[user].append(acc)
            if(user==0)|(user==5)|(user==10)|(user==15)|(user==20):
                print(acc)
        #print(acc_test[cluster_no])
        #print(loss_test[cluster_no])
        #if iter % 1 ==0:
            #print('Round {:3d}, Average loss {:.3f} Test accuracy {:.3f}'.format(iter, loss_avg[cluster_no],acc_test[cluster_no]))
        #print(hist_)
        #print("users in cluster",cluster_no)
        #print(cluster_block[cluster_no])
        #print("Test accuracy of cluster",cluster_no)
        #print(acc_test[cluster_no])
    #print(loss_train)
    sample=sample+2

iteration number 0
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4, 14]
[10, 11, 12, 13]
[20, 21, 22, 23, 24]
2009547350.1472335
tensor(50.)
tensor(50.)
tensor(50.)
tensor(50.)
tensor(50.1500)
iteration number 1
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
2107997570.1492443
tensor(50.)
tensor(50.)
tensor(50.)
tensor(50.)
tensor(53.2000)
iteration number 2
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
2173324699.4608955
tensor(50.)
tensor(49.0500)
tensor(50.)
tensor(50.)
tensor(52.2500)
iteration number 3
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
2253106559.58984
tensor(50.)
tensor(50.3500)
tensor(50.)
tensor(50.)
tensor(50.4000)
iteration number 4
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
2450310516.3989444
tensor(50.)
tensor(50.6000)
tensor(50.)
tensor(50.)
tensor(50.1000)
itera

646000381.3742654
tensor(58.9500)
tensor(79.2000)
tensor(79.9000)
tensor(69.4500)
tensor(72.0500)
iteration number 40
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
656848511.6193309
tensor(61.0500)
tensor(79.7500)
tensor(80.3000)
tensor(69.5500)
tensor(72.2500)
iteration number 41
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
662427578.3769805
tensor(60.3500)
tensor(78.9500)
tensor(80.2500)
tensor(70.2000)
tensor(72.0500)
iteration number 42
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
677905204.323931
tensor(57.7000)
tensor(80.4500)
tensor(80.3000)
tensor(68.0500)
tensor(72.4500)
iteration number 43
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
650287218.8404808
tensor(62.4500)
tensor(80.5500)
tensor(80.5500)
tensor(70.6000)
tensor(72.5000)
iteration number 44
[5, 6, 7, 8, 9]
[15, 16, 17, 18

432485500.44022846
tensor(67.2500)
tensor(84.2000)
tensor(83.1000)
tensor(75.8500)
tensor(76.5500)
iteration number 79
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
405475638.4021222
tensor(67.1000)
tensor(84.2000)
tensor(83.6000)
tensor(75.1500)
tensor(76.5500)
iteration number 80
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
382783061.34765047
tensor(66.6000)
tensor(81.7000)
tensor(83.6000)
tensor(75.3000)
tensor(76.)
iteration number 81
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
382250363.495991
tensor(65.)
tensor(84.3500)
tensor(82.8500)
tensor(76.1500)
tensor(76.7000)
iteration number 82
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
398346607.2067487
tensor(66.8000)
tensor(83.7000)
tensor(83.1000)
tensor(76.8500)
tensor(76.9500)
iteration number 83
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]


[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
221570137.63804507
tensor(73.)
tensor(85.6000)
tensor(85.)
tensor(77.3000)
tensor(78.3000)
iteration number 118
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
245899354.7750926
tensor(71.7500)
tensor(85.6000)
tensor(85.4000)
tensor(77.9500)
tensor(78.8000)
iteration number 119
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
224971867.44481105
tensor(72.9000)
tensor(85.6000)
tensor(85.2000)
tensor(77.6500)
tensor(78.2000)
iteration number 120
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
199597645.7591378
tensor(70.2500)
tensor(85.7500)
tensor(85.2500)
tensor(77.8000)
tensor(77.6500)
iteration number 121
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
224802791.6479502
tensor(72.5500)
tensor(85.9000)
tenso

KeyboardInterrupt: 

In [ ]:
print(sample_size)

In [ ]:
print(len(idx_users[1]))
print(user2)

In [ ]:
print(sum((received)**2))

In [ ]:
user_acc=[]
for i in range(25):
    user_acc.append([])
for i in range(25):
    for j in noise_acc[i]:
        user_acc[i].append(float(j))
print(user_acc[19][60])

In [ ]:
Avg_acc_cluster0=[]
Avg_acc_cluster1=[]
Avg_acc_cluster2=[]
Avg_acc_cluster3=[]
Avg_acc_cluster4=[]
for i in range(500):
    Avg_acc_cluster0.append((user_acc[0][i]+user_acc[1][i]+user_acc[2][i]+user_acc[3][i]+user_acc[4][i])/5)
    Avg_acc_cluster1.append((user_acc[5][i]+user_acc[6][i]+user_acc[7][i]+user_acc[8][i]+user_acc[9][i])/5)
    Avg_acc_cluster2.append((user_acc[10][i]+user_acc[11][i]+user_acc[12][i]+user_acc[13][i]+user_acc[14][i])/5)
    Avg_acc_cluster3.append((user_acc[15][i]+user_acc[16][i]+user_acc[17][i]+user_acc[18][i]+user_acc[19][i])/5)
    Avg_acc_cluster4.append((user_acc[20][i]+user_acc[21][i]+user_acc[22][i]+user_acc[23][i]+user_acc[24][i])/5)

In [ ]:
plt.plot(range(len(Avg_acc_cluster0)), Avg_acc_cluster0)
#plt.ylabel('cluster 0 accuracy(%)')
plt.ylabel('cluster 0 test accuracy')
plt.xlabel('iteration no.')
plt.show()
# plt.savefig('./save/fed_{}_{}_{}_C{}_iid{}.png'.format(args.dataset, args.model, args.epochs, args.frac, args.iid))
plt.plot(range(len(Avg_acc_cluster1)), Avg_acc_cluster1)
#plt.ylabel('cluster 0 accuracy(%)')
plt.ylabel('cluster 1 test accuracy')
plt.xlabel('iteration no.')
plt.show()
plt.plot(range(len(Avg_acc_cluster2)), Avg_acc_cluster2)
#plt.ylabel('cluster 0 accuracy(%)')
plt.ylabel('cluster 2 test accuracy')
plt.xlabel('iteration no.')
plt.show()
plt.plot(range(len(Avg_acc_cluster3)), Avg_acc_cluster3)
#plt.ylabel('cluster 0 accuracy(%)')
plt.ylabel('cluster 3 test accuracy')
plt.xlabel('iteration no.')
plt.show()
plt.plot(range(len(Avg_acc_cluster4)), Avg_acc_cluster4)
#plt.ylabel('cluster 0 accuracy(%)')
plt.ylabel('cluster 4 test accuracy')
plt.xlabel('iteration no.')
plt.show()

In [ ]:
avg_acc=0
#print(sum(noise_acc[20]))
for i in range(25):
    avg_acc=avg_acc+sum(noise_acc[i])/200
#avg_acc=(sum(Avg_acc_cluster0)+sum(Avg_acc_cluster3)+sum(Avg_acc_cluster4))/(216*3)
avg_acc=avg_acc/25
print(avg_acc)

In [ ]:

print("Cluster 0 accuracy")
print(Avg_acc_cluster0)
print("gap")
print("Cluster 1 accuracy")
print(Avg_acc_cluster1)
print("gap")
print("Cluster 2 accuracy")
print(Avg_acc_cluster2)
print("gap")
print("Cluster 3 accuracy")
print(Avg_acc_cluster3)
print("gap")
print("Cluster 4 accuracy")
print(Avg_acc_cluster4)
print("gap")